Train

In [5]:
import pickle
import numpy as np
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
from keras.callbacks import Callback
from sklearn.preprocessing import LabelEncoder

class F1ScoreCallback(Callback):
    def __init__(self, X_val, y_val):
        super(F1ScoreCallback, self).__init__()
        self.X_val = X_val
        self.y_val = y_val
        self.best_f1 = 0.0
        self.best_model = None
        self.f1_scores = []

    def on_epoch_end(self, epoch, logs=None):
        y_val_pred = np.argmax(self.model.predict(self.X_val), axis=1)
        f1 = f1_score(self.y_val, y_val_pred, average='weighted')
        self.f1_scores.append(f1)
        

        if f1 > self.best_f1:
            self.best_f1 = f1
            self.best_model = self.model
            print(f"Epoch {epoch + 1} - F1 Score: {f1:.4f}")
            print("Saved best model")
            print(self.f1_scores)

with open('train_without_test4.pickle', 'rb') as f1:
    balanced = pickle.load(f1)

with open('test_terms_with_embeddings.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

train = np.array([item['cve_terms_ada_embedding'] for item in balanced if item['cwe'] != 'None'])
test = np.array([item['cwe'] for item in balanced if item['cwe'] != 'None'])
np.random.seed(42)
X_train, X_val, y_train, y_val = train_test_split(train,test,test_size=0.1,random_state=42)

X_test = np.array([item['cve_terms_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

label_encoder_train = LabelEncoder()
y_train_encoded = label_encoder_train.fit_transform(y_train)
label_encoder_test = LabelEncoder()
y_test_encoded = label_encoder_test.fit_transform(y_test)


input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))

model = Sequential()
model.add(Dense(128, input_dim=input_dim, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(output_dim, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

f1_callback = F1ScoreCallback(X_val, label_encoder_train.transform(y_val))

history = model.fit(X_train, y_train_encoded, epochs=40, batch_size=32, validation_data=(X_val, label_encoder_train.transform(y_val)), verbose=1, callbacks=[f1_callback])

best_model = f1_callback.best_model


# Save the best model
joblib.dump(best_model, 'CWE_classes.joblib')

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

joblib.dump(label_encoder_train, 'label_encoder_train.joblib')

Epoch 1/40
260/260 [==============================] - 0s 1ms/step loss: 1.8803 - accur
Epoch 1 - F1 Score: 0.5846
Saved best model
[0.5846069226494558]
2336/2336 [==============================] - 8s 3ms/step - loss: 1.8783 - accuracy: 0.5031 - val_loss: 1.4178 - val_accuracy: 0.6076
Epoch 2/40
260/260 [==============================] - 1s 2ms/step loss: 1.3
Epoch 2 - F1 Score: 0.6252
Saved best model
[0.5846069226494558, 0.6251923115572853]
2336/2336 [==============================] - 9s 4ms/step - loss: 1.3175 - accuracy: 0.6293 - val_loss: 1.2906 - val_accuracy: 0.6380
Epoch 3/40
260/260 [==============================] - 1s 2ms/step loss: 1.2
Epoch 3 - F1 Score: 0.6310
Saved best model
[0.5846069226494558, 0.6251923115572853, 0.6310399700337717]
2336/2336 [==============================] - 11s 5ms/step - loss: 1.2113 - accuracy: 0.6553 - val_loss: 1.2355 - val_accuracy: 0.6459
Epoch 4/40
260/260 [==============================] - 1s 2ms/step loss: 1.1
Epoch 4 - F1 Score: 0.6424
Sav

['label_encoder_train.joblib']

Inference

In [2]:
import pickle
import numpy as np
from sklearn.metrics import classification_report
import joblib
from sklearn.preprocessing import LabelEncoder
# Save the best model
with open('test_terms_with_embeddings.pickle', 'rb') as f2:
    unbalanced = pickle.load(f2)

X_test = np.array([item['cve_terms_ada_embedding'] for item in unbalanced if item['cwe'] != 'None'])
y_test = np.array([item['cwe'] for item in unbalanced if item['cwe'] != 'None'])

best_model=joblib.load('best_model.joblib')
label_encoder_train=joblib.load('label_encoder_train.joblib')

# Make predictions on the test set
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

y_pred_original = label_encoder_train.inverse_transform(y_pred)

print("Classification Report:\n", classification_report(y_test, y_pred_original, digits=4))

444/444 [==============================] - 1s 1ms/step
Classification Report:
               precision    recall  f1-score   support

         119     0.8414    0.4561    0.5915      1070
         120     0.3504    0.8724    0.5000       196
         125     0.7726    0.8365    0.8032       532
         134     0.8095    0.8947    0.8500        19
         189     0.6098    0.6303    0.6198       119
         190     0.6803    0.8300    0.7477       200
          20     0.4444    0.1827    0.2590       810
         200     0.5622    0.5746    0.5683       590
         203     0.6061    0.7407    0.6667        27
          22     0.8609    0.8958    0.8780       518
         254     0.1333    0.1176    0.1250        34
         255     0.3247    0.3731    0.3472        67
         264     0.5304    0.4851    0.5067       503
         269     0.3431    0.4434    0.3868       106
         276     0.3111    0.2188    0.2569        64
         284     0.2478    0.2276    0.2373       123
  